In [ ]:
import os 
import numpy as np
import cv2
from glob import glob
from PIL import Image
from keras import backend as K
import imutils
import gc
label_max_len = 32
import re
import string
from collections import Counter

In [ ]:
f_val = open('./data_split/overlap/txt_overlap_val.txt')
txt_overlap_val = []
for line in f_val:
    line = line.strip('\n')
    txt_overlap_val.append(line)
    
f_train = open('./data_split/overlap/txt_overlap_train.txt')
txt_overlap_train = []
for line in f_train:
    line = line.strip('\n')
    txt_overlap_train.append(line)

In [ ]:
def untokenize(words):
    """
    Untokenizing a text undoes the tokenizing operation, restoring
    punctuation and spaces to the places that people expect them to be.
    Ideally, `untokenize(tokenize(text))` should be identical to `text`,
    except for line breaks.
    """
    text = ' '.join(words)
    step1 = text.replace("`` ", '"').replace(" ''", '"').replace('. . .',  '...')
    step2 = step1.replace(" ( ", " (").replace(" ) ", ") ")
    step3 = re.sub(r' ([.,:;?!%]+)([ \'"`])', r"\1\2", step2)
    step4 = re.sub(r' ([.,:;?!%]+)$', r"\1", step3)
    step5 = step4.replace(" '", "'").replace(" n't", "n't").replace(
         "can not", "cannot")
    step6 = step5.replace(" ` ", " '")
    return step6.strip()

# Source: https://stackoverflow.com/questions/367155/splitting-a-string-into-words-and-punctuation
def tokenize(text):
    return re.findall(r"\w+|[^\w\s]", text, re.UNICODE)

def words(text):
    return re.findall(r'\w+', text.lower())

def P(word, N=None):
    "Probability of `word`."
    if N is None:
        N = sum(dictionary.values())
    return dictionary[word] / N

def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in dictionary)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

# Correct words
def corrections(words):
    return [correction(word) for word in words]

# Correct sentence
def sentence(sentence):
    return untokenize(corrections(tokenize(sentence)))

In [ ]:
path = './grid.txt'
dictionary = Counter(list(string.punctuation) + words(open(path).read()))

In [ ]:
for i in range(len(txt_overlap_train)):
    temp = sentence(txt_overlap_train[i])
    txt_overlap_train[i] = temp
    
for i in range(len(txt_overlap_val)):
    temp = sentence(txt_overlap_val[i])
    txt_overlap_val[i] = temp

In [ ]:
# save txt file
with open('./data_split/overlap/correct_txt_overlap_val.txt', 'w') as txt_val:
    for txt in txt_overlap_val:
        txt_val.write(txt)
        txt_val.write('\n')

with open('./data_split/overlap/correct_txt_overlap_train.txt', 'w') as txt_train:
    for txt in txt_overlap_train:
        txt_train.write(txt)
        txt_train.write('\n')